## Library

In [2]:
import os 
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math
from math import sqrt
import pickle 

from collections import Counter
from sklearn.metrics.pairwise import linear_kernel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.linear_model import LinearRegression



## Loading Data 

In [29]:
D = np.load('tf_idf_vector_matrix.npy') # load

In [30]:
D.shape

(9025, 22336)

In [11]:
smd = pd.read_csv('../the-movies-dataset/movies_metadata_equal_ratings.csv')
ratings = pd.read_csv('../the-movies-dataset/ratings_equal_movies_metadata.csv')
train = pd.read_csv('../the-movies-dataset/ratings_train.csv')
test = pd.read_csv('../the-movies-dataset/ratings_test.csv')

In [12]:
N = len(ratings)

In [13]:
smd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9025 entries, 0 to 9024
Data columns (total 26 columns):
adult                    9025 non-null bool
belongs_to_collection    1670 non-null object
budget                   9025 non-null int64
genres                   9025 non-null object
homepage                 1954 non-null object
id                       9025 non-null int64
imdb_id                  9025 non-null object
original_language        9025 non-null object
original_title           9025 non-null object
overview                 9013 non-null object
popularity               9025 non-null float64
poster_path              9022 non-null object
production_companies     9025 non-null object
production_countries     9025 non-null object
release_date             9025 non-null object
revenue                  9025 non-null float64
runtime                  9025 non-null float64
spoken_languages         9025 non-null object
status                   9023 non-null object
tagline             

In [14]:
smd

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,description,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"['Animation', 'Comedy', 'Family']",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ...",1995
1,False,NaN,65000000,"['Adventure', 'Fantasy', 'Family']",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,When siblings Judy and Peter discover an encha...,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"['Romance', 'Comedy']",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,A family wedding reignites the ancient feud be...,1995
3,False,NaN,16000000,"['Comedy', 'Drama', 'Romance']",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"Cheated on, mistreated and stepped on, the wom...",1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,['Comedy'],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Just when George Banks has recovered from his ...,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9020,False,"{'id': 286023, 'name': 'Sharknado Collection',...",0,"['Comedy', 'Horror', 'Science Fiction']",http://www.syfy.com/sharknado4,390989,tt4831420,en,Sharknado 4: The 4th Awakens,The new installment of the Sharknado franchise...,...,85.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"What happens in Vegas, stays in Vegas. Unless ...",Sharknado 4: The 4th Awakens,False,4.3,88.0,The new installment of the Sharknado franchise...,2016
9021,False,NaN,8000000,['Drama'],NaN,159550,tt0255313,en,The Last Brickmaker in America,A man must cope with the loss of his wife and ...,...,85.0,[],Released,NaN,The Last Brickmaker in America,False,7.0,1.0,A man must cope with the loss of his wife and ...,2001
9022,False,NaN,1000000,"['Thriller', 'Romance']",NaN,392572,tt5165344,hi,रुस्तम,"Rustom Pavri, an honourable officer of the Ind...",...,150.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,Decorated Officer. Devoted Family Man. Defendi...,Rustom,False,7.3,25.0,"Rustom Pavri, an honourable officer of the Ind...",2016
9023,False,NaN,15050000,"['Adventure', 'Drama', 'History', 'Romance']",NaN,402672,tt3859980,hi,Mohenjo Daro,"Village lad Sarman is drawn to big, bad Mohenj...",...,155.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,NaN,Mohenjo Daro,False,6.7,26.0,"Village lad Sarman is drawn to big, bad Mohenj...",2016


In [15]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99810 entries, 0 to 99809
Data columns (total 6 columns):
userId        99810 non-null int64
id            99810 non-null float64
rating        99810 non-null float64
movieId       99810 non-null int64
timestamp     99810 non-null int64
movieIndex    99810 non-null int64
dtypes: float64(2), int64(4)
memory usage: 4.6 MB


In [16]:
ratings

,userId,id,rating,movieId,timestamp,movieIndex
0,0,152.0,2.5,1371,1260759135,1107
1,0,9909.0,2.5,31,1260759144,30
2,0,847.0,2.0,2193,1260759198,1739
3,0,9426.0,2.5,2455,1260759113,1958
4,0,6114.0,3.5,1339,1260759125,1079
...,...,...,...,...,...,...
99805,670,786.0,2.0,3897,1063503718,3108
99806,670,7443.0,4.0,3751,1065111939,2995
99807,670,1891.0,5.0,1196,1064890635,949
99808,670,279.0,4.0,1225,1065149143,977


In [17]:
train.shape

(79848, 6)

In [18]:
test.shape

(19962, 6)

In [19]:
n_users = len(ratings['userId'].unique().astype('int'))
n_items = len(ratings['id'].unique().astype('int'))
n_users

671

In [20]:
len(train[train['userId'] == 1])

63

In [21]:
len(test[test['userId'] == 1])

12

In [22]:
ratings_train = train[['userId','id','rating','movieIndex']].values
ratings_test = test[['userId','id','rating','movieIndex']].values

In [23]:
ratings_train.shape

(79848, 4)

In [24]:
ratings_test.shape

(19962, 4)

In [25]:
# this fuction return movie_index in meta_data_dataset and ratings of user  
def get_items_rated_by_user(rate_matrix, user_id):
    y = rate_matrix[:,0] # get all the value in user_id column
    # get list of user_id index in y which value equal input 
    # user_id
    ids = np.where(y == user_id)[0]
    # get list movie_id matching user_id 
    item_ids = rate_matrix[ids, 3]
    item_ids = [int(i) for i in item_ids]
    
    scores = rate_matrix[ids,2]
    return (item_ids, scores)

In [26]:
#a, b = get_items_rated_by_user(ratings_train, 670)
#for i in a:
#    print(i)

In [27]:
D.shape

()

In [28]:
d = D.shape[1] # movie vector dimention
W = np.zeros((d, n_users)) 
b = np.zeros((1,n_users))

IndexError: tuple index out of range

In [ ]:
for i in range(n_users):
        idx, scores = get_items_rated_by_user(ratings_train, i)
        clf = Ridge(alpha=1.0, random_state=241)
        #idx = np.asarray(idx)
        Xhat = D[idx,:]
        
        clf.fit(Xhat, scores)
        W[:, i] = clf.coef_
        b[0, i] = clf.intercept_

In [ ]:
W

In [ ]:
Yhat = D.dot(W) + b
Yhat.shape

In [ ]:
type(Yhat)

In [27]:
n = 0
ids, scores = get_items_rated_by_user(ratings_test, n)
result = Yhat[ids, n]

In [28]:
print("ratings prediction of user ",n," :")
for i in range(len(result)):
    print("movie index : ",ids[i]," - ","predict ratings : ",result[i]," - ","true ratings : ",scores[i])

ratings prediction of user  0  :
movie index :  2920  -  predict ratings :  2.5586603901269434  -  true ratings :  3.0
movie index :  1107  -  predict ratings :  2.552740519031972  -  true ratings :  2.5
movie index :  1704  -  predict ratings :  2.553088118624569  -  true ratings :  3.0
movie index :  927  -  predict ratings :  2.5599975682975105  -  true ratings :  4.0
movie index :  1136  -  predict ratings :  2.5569424868364607  -  true ratings :  1.0
movie index :  2375  -  predict ratings :  2.5636169940256095  -  true ratings :  1.0


In [29]:
#train[train['userId'] == 0]

In [30]:
def evaluate(Yhat, rates, W, b):
    se = 0
    cnt = 0
    for n in range(n_users):
        ids, scores_truth = get_items_rated_by_user(rates, n)
        scores_pred = Yhat[ids, n]
        e = scores_truth - scores_pred 
        se += (e*e).sum(axis = 0)
        cnt += e.size 
    return sqrt(se/cnt)

In [31]:
print('RMSE for train case:', evaluate(Yhat, ratings_train, W, b))
print('RMSE for test case:', evaluate(Yhat, ratings_test, W, b))

RMSE for train case: 0.7663848410323987
RMSE for test case: 0.9540124789544039


In [32]:
test

,userId,id,rating,movieId,timestamp,movieIndex
0,0,11072.0,3.0,3671,1260759117,2920
1,0,152.0,2.5,1371,1260759135,1107
2,0,8393.0,3.0,2150,1260759194,1704
3,0,11216.0,4.0,1172,1260759205,927
4,0,3179.0,1.0,1405,1260759203,1136
...,...,...,...,...,...,...
19957,670,8346.0,3.0,5299,1065112004,4043
19958,670,824.0,3.5,4308,1065111985,3413
19959,670,51927.0,4.0,6269,1065149201,4538
19960,670,745.0,4.0,2762,1064891036,2208


In [33]:
#np.save('tf_idf_pred_rating.npy', Yhat)

In [34]:
predict_rating = []
for n in range(n_users):
    ids, scores_truth = get_items_rated_by_user(ratings_test, n)
    for i in ids:
        scores_pred = Yhat[i, n]
        predict_rating.append(scores_pred)           
        


In [35]:
# this fuction return movie_index in meta_data_dataset and ratings of user  
def get_items_rated_by_user(rate_matrix, user_id):
    y = rate_matrix[:,0] # get all the value in user_id column
    # get list of user_id index in y which value equal input 
    # user_id
    ids = np.where(y == user_id)[0]
    # get list movie_id matching user_id 
    item_ids = rate_matrix[ids, 3]
    item_ids = [int(i) for i in item_ids]
    
    scores = rate_matrix[ids,2]
    return (item_ids, scores)

In [36]:
def get_list_movie_have_not_rated(rate_matrix, userId,movie_count):
    user_rated, score = get_items_rated_by_user(rate_matrix,userId)
    result = []
    for i in range(movie_count):
         if i not in user_rated: result.append(i)
    return (userId, result)            

In [38]:
not_rates = {} 
for i in range(671): 
        userId, result = get_list_movie_have_not_rated(ratings_train,i,N)
        not_rates[userId] = result 

In [50]:
for i in not_rates:
    for j in i:
        print(j)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [33]:
f = open('tf_idf_pred_rating.pckl', 'wb')
pickle.dump(predict_rating, f)
f.close()

In [34]:
test['TF-IDF pred'] = predict_rating

In [35]:
test

,userId,id,rating,movieId,timestamp,movieIndex,TF-IDF pred
0,0,9909.0,2.5,31,1260759144,30,2.426854
1,0,847.0,2.0,2193,1260759198,1739,2.423113
2,0,97.0,4.0,2105,1260759139,1661,2.419296
3,0,11216.0,4.0,1172,1260759205,927,2.431504
4,0,1103.0,2.0,1129,1260759185,903,2.421217
...,...,...,...,...,...,...,...
19957,670,11310.0,2.5,432,1063503739,381,3.872884
19958,670,712.0,3.0,357,1063503998,322,3.845205
19959,670,630.0,4.0,919,1065149458,737,3.941357
19960,670,687.0,4.0,36,1065149314,34,3.878385


In [36]:
Yhat[381,670]

3.8728841163963232